In [1]:
import requests
import pandas as pd
import time

import pandas as pd
from os import listdir
from os.path import isfile, join

import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model, load_model
import pandas as pd
import numpy as np
from os import listdir      
from os.path import isfile, join


In [2]:
# symbols and technical indicators [code, interval, name]
# https://www.alphavantage.co/documentation/#technical-indicators
#
# got rid of JNK (weirdly high open z-score mean), HYG (weirdly low low z-score mean), and EWZ/IEF (infinite end values)

symbol_list = ['SPY','QQQ','XLF','EEM','XLE','SLV','FXI','GDX','EFA','TLT','LQD','XLU','XLV','XLI','XLK','IEF','XLB','JETS','BND']
#symbol_list = ['SPY','QQQ']
tech_list = [['SMA',50,'Technical Analysis: SMA'],
             ['EMA',21,'Technical Analysis: EMA'],
             ['RSI',14,'Technical Analysis: RSI']]

print(f"This will take about {len(symbol_list) * (len(tech_list) + 1)*15/60} minutes to run.")

for symbol in symbol_list:

    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey=PDS8Y8E8KULJVDET"
    r = requests.get(url)
    data = r.json()
    df_price = pd.DataFrame(data['Time Series (Daily)']).T
    time.sleep(15)

    for tech in tech_list:
        url = f"https://www.alphavantage.co/query?function={tech[0]}&symbol={symbol}&interval=daily&time_period={tech[1]}&series_type=close&apikey=PDS8Y8E8KULJVDET"
        r = requests.get(url)
        data = r.json()
        df_tech = pd.DataFrame(data[tech[2]]).T
        df_price = df_price.merge(df_tech, how='inner', left_index=True, right_index=True)
        time.sleep(15)
    
    df_price.to_csv(f"../data/predict/{symbol}_daily.csv")
    print(f"{symbol} saved")


This will take about 19.0 minutes to run.
SPY saved
QQQ saved
XLF saved
EEM saved
XLE saved
SLV saved
FXI saved
GDX saved
EFA saved
TLT saved
LQD saved
XLU saved
XLV saved
XLI saved
XLK saved
IEF saved
XLB saved
JETS saved
BND saved


In [2]:
window_sizes = [20,320]

def zscore(x, window):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

In [3]:
onlyfiles = [f for f in listdir('../data/predict/') if isfile(join('../data/predict/', f))]
onlyfiles = list(filter(lambda thisfilename: '_daily.csv' in thisfilename, onlyfiles))

ticker_list = []
ticker_stats_mean = pd.DataFrame()
ticker_stats_std = pd.DataFrame()

for window_size in window_sizes:
    for filename in onlyfiles:
        ticker = filename.split('_')[0]
        ticker_list.append(ticker)
        
        # import csv, sort by date so percent change works, drop unneeded columns,
        # rename columns, calculate moving averages, calulate percent changes, drop na's
        df = pd.read_csv(f"../data/predict/{ticker}_daily.csv")
        df.sort_index(inplace=True, ascending=False)
        df = df.drop(['5. adjusted close', '7. dividend amount', '8. split coefficient','SMA','EMA'], axis=1)
        df.columns = ['date','open','high','low','close','volume','rsi']

        # moving averages, convert to percentage of close
        df['sma'] = df.iloc[:,3].rolling(window=50).mean()/df.iloc[:,3]
        df['ema'] = df.iloc[:,3].ewm(span=21).mean()/df.iloc[:,3]

        # percent change
        df['open'] = df['open'].pct_change()
        df['high'] = df['high'].pct_change()
        df['low'] = df['low'].pct_change()
        df['close'] = df['close'].pct_change()
        df['volume'] = df['volume'].pct_change()
        df = df.dropna()

        # zscore
        df['open'] =zscore(df['open'], window=window_size)
        df['high'] = zscore(df['high'], window=window_size)
        df['low'] = zscore(df['low'], window=window_size)
        df['close'] = zscore(df['close'], window=window_size)
        df['volume'] = zscore(df['volume'], window=window_size)
        df['rsi'] = zscore(df['rsi'], window=window_size)
        df['sma'] = zscore(df['sma'], window=window_size)
        df['ema'] = zscore(df['ema'], window=window_size)
        df = df.dropna()

        # write data and describe to csv
        df.to_csv(f"../data/predict/{ticker}_{window_size}_processed.csv")

ticker_list = set(ticker_list)

In [4]:
stats_dict = {}

# iterating through the window sizes provided in the list above
for window_size in window_sizes:
    print(f"starting window size: {window_size}")
    
    # iterating through the list of processed files
    for ticker in ticker_list:
        # empty dataframe for this ticker's flattened data
        flattened_df = pd.DataFrame()
        df = pd.read_csv(f"../data/predict/{ticker}_{window_size}_processed.csv").drop(['Unnamed: 0'], axis=1)
        
        # iterating through grouped rows (window size)
        for i in range(df.shape[0]-window_size-1, df.shape[0]-window_size+1):
            # resetting the index each time so column names align
            df_window = df.iloc[i:i+window_size,].reset_index(drop=True)
            # stats dict
            if i == (df.shape[0]-window_size):
                stats_dict[ticker] = {'open': {'mean': df_window['open'].mean(), 'std': df_window['open'].std()},
                                      'high': {'mean': df_window['high'].mean(), 'std': df_window['high'].std()},
                                      'low': {'mean': df_window['low'].mean(), 'std': df_window['low'].std()},
                                      'close': {'mean': df_window['close'].mean(), 'std': df_window['close'].std()}}
            # mapping the index as a string so it can be concatenated
            df_window.index = df_window.index.map(str)
            # unstacking the window to one row
            df_window = df_window.unstack().to_frame().sort_index(level=1).T
            # renaming the columns
            df_window.columns = df_window.columns.map('_'.join)
            # concatenating the flattened row to the dataframe
            flattened_df = pd.concat([flattened_df, df_window], axis=0)
            
                
        # writing the flattened dataframe for this ticker and window size to csv
        flattened_df.to_csv(f"../data/predict/{ticker}_{window_size}_flattened.csv")
        #print(f"{ticker} {window_size} flattened")

starting window size: 20
starting window size: 320


In [ ]:
forecast_dict = {}

for ticker in ticker_list:

    forecast_dict[ticker] = {}

    for window_size in [20, 320]:

        onlyfiles = [f for f in listdir('./results') if isfile(join('./results', f))]
        onlyfiles = list(filter(lambda thisfilename: f"-{window_size}-" in thisfilename, onlyfiles))

        # import csv, sort by date so percent change works, drop unneeded columns,
        # rename columns, calculate moving averages, calulate percent changes, drop na's
        df = pd.read_csv(f"../data/predict/{ticker}_{window_size}_flattened.csv")
        df_daily = pd.read_csv(f"../data/predict/{ticker}_daily.csv")
        df_daily = df_daily.drop(['5. adjusted close', '6. volume', '7. dividend amount', '8. split coefficient','SMA','EMA','RSI'], axis=1)
        df_daily.columns = ['date','open','high','low','close']
        print(f"Latest date that data is available for in ../data/predict/{ticker}_{window_size}_flattened.csv is {df_daily.date[0]} (should be latest trading day).")

        # features to use
        items = ["close", "ema", "high", "low", "open", "rsi", "sma", "volume"]
        day_counts = [f"_{i}" for i in range(1, window_size)]
        FEATURE_COLUMNS = []
        for day_count in day_counts:
            for item in items:
                FEATURE_COLUMNS.append(f"{item}{day_count}")

        df_pred = df[FEATURE_COLUMNS]

        # Get samples to predict
        samples_to_predict = np.array(df_pred)

        # Convert into Numpy array
        samples_to_predict = samples_to_predict.reshape((samples_to_predict.shape[0], 1, samples_to_predict.shape[1]))

        # File path
        for model_filepath in onlyfiles:
            print(f"Starting {model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]} for {ticker}.")
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]] = {'window_size': window_size, 'tomorrow_forecast_open': None, 'tomorrow_forecast_high': None, 'tomorrow_forecast_low': None, 'tomorrow_forecast_close': None,
                                    'today_forecast_open': None, 'today_forecast_high': None, 'today_forecast_low': None, 'today_forecast_close': None,
                                    'today_actual_open': None, 'today_actual_high': None, 'today_actual_low': None, 'today_actual_close': None, 
                                    'today_error_open': None, 'today_error_high': None, 'today_error_low': None, 'today_error_close': None}
            model = load_model(f"./results/{model_filepath}", compile=True)
            # Predict
            predictions = model.predict(samples_to_predict)
            
            linear_df = pd.DataFrame(predictions)
            linear_df.columns = ['close','high','low','open']
            linear_df.index = ['today','tomorrow']

            linear_df.close = linear_df.close*stats_dict[ticker]['close']['std'] + stats_dict[ticker]['close']['mean']
            linear_df.high = linear_df.high*stats_dict[ticker]['high']['std'] + stats_dict[ticker]['high']['mean']
            linear_df.low = linear_df.low*stats_dict[ticker]['low']['std'] + stats_dict[ticker]['low']['mean']
            linear_df.open = linear_df.open*stats_dict[ticker]['open']['std'] + stats_dict[ticker]['open']['mean']

            linear_df = linear_df[['open','high','low','close']]
            
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['tomorrow_forecast_open'] = df_daily[['open']].iloc[0,][0] + (df_daily[['open']].iloc[0,][0] * (linear_df[['open']].iloc[1,][0]/100))
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['tomorrow_forecast_high'] = df_daily[['high']].iloc[0,][0] + (df_daily[['high']].iloc[0,][0] * (linear_df[['high']].iloc[1,][0]/100))
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['tomorrow_forecast_low'] = df_daily[['low']].iloc[0,][0] + (df_daily[['low']].iloc[0,][0] * (linear_df[['low']].iloc[1,][0]/100))
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['tomorrow_forecast_close'] = df_daily[['close']].iloc[0,][0] + (df_daily[['close']].iloc[0,][0] * (linear_df[['close']].iloc[1,][0]/100))
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['tomorrow_forecast_open_pct'] = ((df_daily[['open']].iloc[0,][0] + (df_daily[['open']].iloc[0,][0] * (linear_df[['open']].iloc[1,][0]/100))) - df_daily[['close']].iloc[0,][0])/df_daily[['close']].iloc[0,][0] * 100
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['tomorrow_forecast_high_pct'] = ((df_daily[['high']].iloc[0,][0] + (df_daily[['high']].iloc[0,][0] * (linear_df[['high']].iloc[1,][0]/100))) - df_daily[['close']].iloc[0,][0])/df_daily[['close']].iloc[0,][0] * 100
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['tomorrow_forecast_low_pct'] = ((df_daily[['low']].iloc[0,][0] + (df_daily[['low']].iloc[0,][0] * (linear_df[['low']].iloc[1,][0]/100))) - df_daily[['close']].iloc[0,][0])/df_daily[['close']].iloc[0,][0] * 100
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['tomorrow_forecast_close_pct'] = ((df_daily[['close']].iloc[0,][0] + (df_daily[['close']].iloc[0,][0] * (linear_df[['close']].iloc[1,][0]/100))) - df_daily[['close']].iloc[0,][0])/df_daily[['close']].iloc[0,][0] * 100
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_forecast_open'] = df_daily[['open']].iloc[1,][0] + (df_daily[['open']].iloc[1,][0] * (linear_df[['open']].iloc[0,][0]/100))
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_forecast_high'] = df_daily[['high']].iloc[1,][0] + (df_daily[['high']].iloc[1,][0] * (linear_df[['high']].iloc[0,][0]/100))
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_forecast_low'] = df_daily[['low']].iloc[1,][0] + (df_daily[['low']].iloc[1,][0] * (linear_df[['low']].iloc[0,][0]/100))
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_forecast_close'] = df_daily[['close']].iloc[1,][0] + (df_daily[['close']].iloc[1,][0] * (linear_df[['close']].iloc[0,][0]/100))
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_open'] = df_daily[['open']].iloc[0,][0]
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_high'] = df_daily[['high']].iloc[0,][0]
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_low'] = df_daily[['low']].iloc[0,][0]
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_close'] = df_daily[['close']].iloc[0,][0]
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_error_open'] = abs((forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_forecast_open'] - forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_open']) / forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_open'] * 100)
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_error_high'] = abs((forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_forecast_high'] - forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_high']) / forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_high'] * 100)
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_error_low'] = abs((forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_forecast_low'] - forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_low']) / forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_low'] * 100)
            forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_error_close'] = abs((forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_forecast_close'] - forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_close']) / forecast_dict[ticker][model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]]['today_actual_close'] * 100)
            
            print(f"Finished {model_filepath.split('-')[7] + '-' + model_filepath.split('-')[3]} for {ticker}.")
            tf.keras.backend.clear_session()

    pd.DataFrame(forecast_dict[ticker]).to_csv(f"../data/predict_tomorrow/{ticker}_{df_daily.date[0]}_daily_forecast.csv")

Error: Session cannot generate requests